### 1. import dataset and load sample

In [108]:
from datasets import load_dataset

dataset = load_dataset("bavard/personachat_truecased", "sample")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['personality', 'candidates', 'history', 'conv_id', 'utterance_idx'],
        num_rows: 14
    })
    validation: Dataset({
        features: ['personality', 'candidates', 'history', 'conv_id', 'utterance_idx'],
        num_rows: 15
    })
})


In [109]:
print(dataset["train"][0])
    

{'personality': ['I like to remodel homes.', 'I like to go hunting.', 'I like to shoot a bow.', 'My favorite holiday is halloween.'], 'candidates': ['My mom was single with 3 boys, so we never left the projects.', 'I try to wear all black every day. It makes me feel comfortable.', 'Well nursing stresses you out so I wish luck with sister.', 'Yeah just want to pick up Nba nfl getting old.', 'I really like Celine Dion. What about you?', 'No. I live near farms.', "I wish I had a daughter, I'm a boy mom. They're beautiful boys though still lucky.", 'Yeah when I get bored I play gone with the wind my favorite movie.', "Hi how are you? I'm eating dinner with my hubby and 2 kids.", 'Were you married to your high school sweetheart? I was.', 'That is great to hear! Are you a competitive rider?', "Hi, I'm doing ok. I'm a banker. How about you?", "I'm 5 years old.", 'Hi there. How are you today?', 'I totally understand how stressful that can be.', 'Yeah sometimes you do not know what you are actu

### 2. Define history conversation, response, and persona

In [110]:
#select conversation with certain id
conv_id = 6
dataset = load_dataset("bavard/personachat_truecased", "full")
dialog = dataset.filter(lambda example:example['conv_id']== conv_id)


In [111]:
#select persona 
persona = dialog['train']['personality'][-1]
print("User Persona: ", persona, "\n")

#select the history conversation
history_convo = dialog['train']['history'][-1]
print("History Conversation: ", history_convo, "\n")

#select the response
usr_response = dialog['train']['candidates'][-1][-1]
print("Response: ",usr_response)


User Persona:  ['I have a boxer dog.', 'I like baths.', 'I like to listen to music.', 'My father lives in China.'] 

History Conversation:  ["Rock on, I'm listening to my favorite band guns and roses.", 'No kidding? I was just listening to the same thing while taking a bath.', 'Of course. I love to listen to rock.', 'Man my boxer just peed on the carpet!', "Well I'm into black everything. So at least it wouldn't show on my black carpet.", 'Ll. I love black too! Guess I was playing my music too loud.', "I've a black car, purse, wear all black.", 'Maybe I can borrow something as I am packing to visit my dad in China.', 'Wow, does he live there or work?', 'Live. Moved there about ten years ago for a computer tech job.', 'Have you visited him there before?', 'Once. You cannot even throw a gum wrapper or you can get arrested.', 'Sounds a bit scary. I ve never been.'] 

Response:  Well not too much crime there, but a lot of people.


### 3. Preprocess the text to make it suitable for prompting

In [125]:
#preprocess the persona
persona_processed = "User Persona: \n"
for sen in persona:
    persona_processed += sen + "\n"

#preprocess the history conversation
history_convo_processed = "Dialogue history: \n"

#concat all history except the last utter from the bot
for i in range(0,len(history_convo)-1,2):
    bot_uttr = "Bot: " + history_convo[i]
    user_uttr = "User: " + history_convo[i+1]
    full_uttr = bot_uttr + "\n" + user_uttr + "\n"
    history_convo_processed += full_uttr

#add the last utter from bot
history_convo_processed += "Bot: " + history_convo[-1] + "\n"

#concat prompt
prompt = persona_processed + "\n" + history_convo_processed + "\nResponse\nUser:"
print(prompt)

Dialogue history: 
Bot: Rock on, I'm listening to my favorite band guns and roses.
User: No kidding? I was just listening to the same thing while taking a bath.
Bot: Of course. I love to listen to rock.
User: Man my boxer just peed on the carpet!
Bot: Well I'm into black everything. So at least it wouldn't show on my black carpet.
User: Ll. I love black too! Guess I was playing my music too loud.
Bot: I've a black car, purse, wear all black.
User: Maybe I can borrow something as I am packing to visit my dad in China.
Bot: Wow, does he live there or work?
User: Live. Moved there about ten years ago for a computer tech job.
Bot: Have you visited him there before?
User: Once. You cannot even throw a gum wrapper or you can get arrested.
Bot: Sounds a bit scary. I ve never been.

Response
User:


In [126]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = 'sk-proj-zgpS1dH01GeQiQzhICjNT3BlbkFJq8oC1i8atPEckRUxahYK'
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # Specify the model
    messages=[
        {"role": "system", "content": "According to the history conversation provided, pretend to speak like a human user and generate a response."},
        {"role": "user", "content": prompt}
    ],
    temperature=0,  # Adjust based on how creative you want the AI to be
    max_tokens=150, # Adjust based on how long you expect responses to be
)


In [127]:
import json
print(json.dumps(json.loads(response.model_dump_json()), indent=4))


{
    "id": "chatcmpl-9ikTtLOpcukHYtnHPaUuqlCtz9enM",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "China is definitely a unique experience with its strict rules. It's important to be mindful of the cultural differences when visiting.",
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1720452133,
    "model": "gpt-3.5-turbo-0125",
    "object": "chat.completion",
    "service_tier": null,
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 24,
        "prompt_tokens": 233,
        "total_tokens": 257
    }
}


In [128]:
response_chatgpt = response.choices[0].message.content

print(response_chatgpt)

print(usr_response)

China is definitely a unique experience with its strict rules. It's important to be mindful of the cultural differences when visiting.
Well not too much crime there, but a lot of people.


In [273]:
from datasets import load_dataset
import random

def get_random_conv_id(pool, num_conv):
    random_values = random.sample(pool, num_conv)

    for value in random_values:
        pool.remove(value)
    
    return random_values

def extract_format_data(dataset, conv_id):
    dialog = dataset.filter(lambda example:example['conv_id']== conv_id)

    persona = dialog['train']['personality'][-1]
    history_convo = dialog['train']['history'][-1]
    usr_response = dialog['train']['candidates'][-1][-1]

    #preprocess the persona
    persona_processed = "User Persona: \n"
    for sen in persona:
        persona_processed += sen + "\n"

    #preprocess the history conversation
    history_convo_processed = "Dialogue history: \n"

    #concat all history except the last utter from the bot
    for i in range(0,len(history_convo)-1,2):
        bot_uttr = "Bot: " + history_convo[i]
        user_uttr = "User: " + history_convo[i+1]
        full_uttr = bot_uttr + "\n" + user_uttr + "\n"
        history_convo_processed += full_uttr

    #add the last utter from bot
    history_convo_processed += "Bot: " + history_convo[-1] + "\n"

    #preprocess the user response
    usr_response_processed = "Response\nUser: " + usr_response + "\n"
    return [persona_processed, history_convo_processed, usr_response_processed]

def create_example(dataset, conv_id, implicit=False):
    materials = extract_format_data(dataset, conv_id)
    if implicit:
        return materials[1] + materials[2]
    else:
        return materials[0] + materials[1] + materials[2]
    return example

def create_few_shot_examples(dataset, conv_id, few_shot_no, implicit=False):
    max_conv_id = dataset['train']['conv_id'][-1]
    pool = [num for num in range(1, max_conv_id + 1) if num != conv_id]
    random_conv_ids = get_random_conv_id(pool, few_shot_no)

    few_shot_examples = ""
    for index, conv_id in enumerate(random_conv_ids):
        few_shot_examples += f"Demo {index}:\n" + create_example(dataset, conv_id, implicit) + "\n"
    return few_shot_examples


def construct_prompt(dataset, conv_id, prompt_type, few_shot_no):
    max_conv_id = dataset['train']['conv_id'][-1]
    materials = extract_format_data(dataset, conv_id)
    system_prompt = ""
    user_prompt = ""
    few_shot_examples = ""

    if prompt_type == "context_only":
        user_prompt += materials[1] + "User:"

    if prompt_type == "task_prompt_context_implicit":
        system_prompt += "According to the history conversation provided, pretend to speak like a human user and generate a response."
        user_prompt += materials[1] + "User:"

    if prompt_type == "task_prompt_context_explicit":
        system_prompt += "According to the persona and history conversation provided, pretend to speak like a human user and generate a response."
        user_prompt += materials[0]+ materials[1] + "User:"

    if prompt_type == "few_shot_implicit":
        system_prompt += "According to the few shot demos provided, pretend to speak like User and generate a response matching the context."

        few_shot_examples = create_few_shot_examples(dataset, conv_id, few_shot_no,implicit=True)
        user_prompt += few_shot_examples + materials[1]+ "User:"

    return system_prompt, user_prompt, materials[2]

In [277]:
dataset = load_dataset("bavard/personachat_truecased", "full")
system_p, user_p, target_response = construct_prompt(dataset, 6, "few_shot_implicit", 1)

Filter: 100%|██████████| 7801/7801 [00:00<00:00, 52893.94 examples/s]


In [278]:
system_p

'According to the few shot demos provided, pretend to speak like User and generate a response matching the context.'

In [283]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = 'sk-proj-zgpS1dH01GeQiQzhICjNT3BlbkFJq8oC1i8atPEckRUxahYK'
client = OpenAI()

def prompt_chatgpt(prompt_type, conv_id, few_shot_no=0):
    dataset = load_dataset("bavard/personachat_truecased", "full")
    system_prompt, user_prompt, target_response= construct_prompt(dataset, conv_id, prompt_type, few_shot_no)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",  # Specify the model
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.9,  # Adjust based on how creative you want the AI to be
        max_tokens=15, # Adjust based on how long you expect responses to be
    )
    return response.choices[0].message.content, user_prompt, target_response



In [284]:
response, user_prompt, target_repsonse= prompt_chatgpt("task_prompt_context_explicit", 6)

In [287]:
print(user_prompt)
print(response)
print(target_repsonse)

User Persona: 
I have a boxer dog.
I like baths.
I like to listen to music.
My father lives in China.
Dialogue history: 
Bot: Rock on, I'm listening to my favorite band guns and roses.
User: No kidding? I was just listening to the same thing while taking a bath.
Bot: Of course. I love to listen to rock.
User: Man my boxer just peed on the carpet!
Bot: Well I'm into black everything. So at least it wouldn't show on my black carpet.
User: Ll. I love black too! Guess I was playing my music too loud.
Bot: I've a black car, purse, wear all black.
User: Maybe I can borrow something as I am packing to visit my dad in China.
Bot: Wow, does he live there or work?
User: Live. Moved there about ten years ago for a computer tech job.
Bot: Have you visited him there before?
User: Once. You cannot even throw a gum wrapper or you can get arrested.
Bot: Sounds a bit scary. I ve never been.
User:
It can be a bit intimidating at times, but overall it's a fascinating
Response
User: Well not too much crim

In [286]:
response

"It can be a bit intimidating at times, but overall it's a fascinating"

In [167]:
max_conv_id = len(dataset['train']['conv_id'])
print(max_conv_id)

131438


In [154]:
print(example)

User Persona: 
I have a boxer dog.
I like baths.
I like to listen to music.
My father lives in China.
Dialogue history: 
Bot: Rock on, I'm listening to my favorite band guns and roses.
User: No kidding? I was just listening to the same thing while taking a bath.
Bot: Of course. I love to listen to rock.
User: Man my boxer just peed on the carpet!
Bot: Well I'm into black everything. So at least it wouldn't show on my black carpet.
User: Ll. I love black too! Guess I was playing my music too loud.
Bot: I've a black car, purse, wear all black.
User: Maybe I can borrow something as I am packing to visit my dad in China.
Bot: Wow, does he live there or work?
User: Live. Moved there about ten years ago for a computer tech job.
Bot: Have you visited him there before?
User: Once. You cannot even throw a gum wrapper or you can get arrested.
Bot: Sounds a bit scary. I ve never been.
Response:
User: Well not too much crime there, but a lot of people.


In [245]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')

In [250]:
scores['rouge1'].fmeasure

0.7058823529411765

In [252]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def p_cover(persona, response):
    # Tokenize and remove stop words
    persona_tokens = [word for word in nltk.word_tokenize(persona.lower()) if word.isalnum() and word not in stop_words]
    response_tokens = [word for word in nltk.word_tokenize(response.lower()) if word.isalnum() and word not in stop_words]
    
    # Calculate TF-IDF for the persona terms
    vectorizer = TfidfVectorizer(vocabulary=set(persona_tokens))
    tfidf_matrix = vectorizer.fit_transform([response])
    
    # Extract the TF-IDF scores for the persona terms in the response
    tfidf_scores = tfidf_matrix.toarray()[0]
    
    # Calculate the P-Cover score
    p_cover_score = sum(tfidf_scores) / len(persona_tokens)
    
    return p_cover_score

# Example usage
persona = "I love playing basketball and enjoy watching movies."
response = "Basketball is my favorite sport. I also love watching movies in my free time."

p_cover_score = p_cover(persona, response)
print(f"P-Cover Score: {p_cover_score}")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zarius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/zarius/nltk_data'
    - '/Users/zarius/miniconda3/envs/llm_persona/nltk_data'
    - '/Users/zarius/miniconda3/envs/llm_persona/share/nltk_data'
    - '/Users/zarius/miniconda3/envs/llm_persona/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [258]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


# implementation of P-Cover
def calculate_p_cover(persona, response):
    return 0

# implementation of BLEU-1, BLEU-2, BLEU-3, and BLEU-4
def calculate_bleu(reference_sentence, candidate_sentence):
    reference = [reference_sentence.split()]
    candidate = candidate_sentence.split()
    
    # Weights for BLEU-1, BLEU-2, BLEU-3, and BLEU-4
    weights = [
        (1.0, 0, 0, 0),        
        (0.5, 0.5, 0, 0),        
        (1.0 / 3, 1.0 / 3, 1.0 / 3, 0), 
        (0.25, 0.25, 0.25, 0.25) 
    ]
    
    # Use smoothing to handle cases with few n-grams
    smoothing_function = SmoothingFunction()
    scores = sentence_bleu(reference, candidate, weights, smoothing_function=smoothing_function.method1)
    return scores

# rouge1, rouge2, rougeL
def calculate_rouge(reference_sentence, candidate_sentence):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_sentence, candidate_sentence)
    return scores

def cosine_similarity_embeddings(sentence1, sentence2, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode([sentence1, sentence2])
    cos_sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    return cos_sim

def distinct_1(sentence):
    words = sentence.split()
    unique_unigrams = set(words)
    total_unigrams = len(words)
    if total_unigrams == 0:
        return 0.0
    distinct_1_score = len(unique_unigrams) / total_unigrams
    return distinct_1_score

def distinct_2(sentence):
    words = sentence.split()
    bigrams = list(nltk.bigrams(words))
    unique_bigrams = set(bigrams)
    total_bigrams = len(bigrams)
    if total_bigrams == 0:
        return 0.0
    distinct_2_score = len(unique_bigrams) / total_bigrams
    return distinct_2_score

def calculate_metrics(reference_sentence, candidate_sentence):
    bleu_score = calculate_bleu(reference_sentence, candidate_sentence)
    rouge_scores = calculate_rouge(reference_sentence, candidate_sentence)
    cosine_similarity = cosine_similarity_embeddings(reference_sentence, candidate_sentence)
    distinct_1_score = distinct_1(candidate_sentence)
    distinct_2_score = distinct_2(candidate_sentence)

    return {
        'BLEU-1': bleu_score[0],
        'BLEU-2': bleu_score[1],
        'BLEU-3': bleu_score[2],
        'BLEU-4': bleu_score[3],
        'ROUGE-1': rouge_scores['rouge1'].fmeasure,
        'ROUGE-2': rouge_scores['rouge2'].fmeasure,
        'ROUGE-L': rouge_scores['rougeL'].fmeasure,
        'Cosine Similarity': cosine_similarity,
        'Distinct-1': distinct_1_score,
        'Distinct-2': distinct_2_score
    }


In [259]:
reference_sentence = "the cat is on the mat"
candidate_sentence = "the cat is on mat"
calculate_metrics(reference_sentence, candidate_sentence)

{'BLEU-1': 0.8187307530779819,
 'BLEU-2': 0.7090416310250969,
 'BLEU-3': 0.6498270293573523,
 'BLEU-4': 0.5789300674674098,
 'ROUGE-1': 0.9090909090909091,
 'ROUGE-2': 0.6666666666666665,
 'ROUGE-L': 0.9090909090909091,
 'Cosine Similarity': 0.9925266,
 'Distinct-1': 1.0,
 'Distinct-2': 1.0}

In [261]:
num_stories = 10
prompts = ["Once upon a time,"] * num_stories
stories = [""] * len(prompts)

In [262]:
stories

['', '', '', '', '', '', '', '', '', '']

In [272]:
from openai import OpenAI

client = OpenAI()

def prompt_chatgpt_batch(prompt_type, conv_ids, few_shot_no):
    dataset = load_dataset("bavard/personachat_truecased", "full")
    
    system_prompts = []
    user_prompts = []
    target_responses = []

    # Collect prompts for all conversation IDs
    for conv_id in conv_ids:
        system_prompt, user_prompt, target_response = construct_prompt(dataset, conv_id, prompt_type, few_shot_no)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt)
        target_responses.append(target_response)
    
    # Create batch messages for the API request
    messages = [{"role": "system", "content": system_prompt} for system_prompt in system_prompts]
    messages += [{"role": "user", "content": user_prompt} for user_prompt in user_prompts]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=0.9,
        max_tokens=15,  # since persona-chat sets a maximum of 15 words per message
    )
    
    # Process responses and match to the original prompts
    responses = ["" for _ in conv_ids]
    for i, choice in enumerate(response.choices):
        index = i % len(conv_ids)
        responses[index] = choice.message.content
    
    # Combine responses with user prompts and target responses
    batch_responses = []
    for i in range(len(conv_ids)):
        batch_responses.append({
            "response": responses[i],
            "target_response": target_responses[i],
            "user_prompt": user_prompts[i]
        })
    
    return batch_responses

# Example usage:
conv_ids = [1]  # Replace with actual conversation IDs
prompt_type = "context_only"
few_shot_no = 0

batch_results = prompt_chatgpt_batch(prompt_type, conv_ids, few_shot_no)
for result in batch_results:
    print(f"User Prompt: {result['user_prompt']}")
    print(f"Response: {result['response']}")
    print(f"Target Response: {result['target_response']}")
    print()

User Prompt: Dialogue history: 
Bot: Hi, how are you doing today?
User: I am spending time with my 4 sisters what are you up to.
Bot: Wow, four sisters. Just watching game of thrones.
User: That is a good show I watch that while drinking iced tea.
Bot: I agree. What do you do for a living?
User: I'm a researcher I'm researching the fact that mermaids are real.
Bot: Interesting. I'm a website designer. Pretty much spend all my time on the computer.
User: That's cool my mom does the same thing.
Bot: That's awesome. I have always had a love for technology.
User: Tell me more about yourself.
Bot: I really enjoy free diving, how about you, have any hobbies?
User: I enjoy hanging with my mother she's my best friend.
Bot: That's nice. Moms are pretty cool too.
User:
Response: Yes, they are. I'm really lucky to have such a close relationship
Target Response: Response
User: I'm also fascinated with mermaids.




In [271]:
print(batch_results[0]['response'])

Dialogue history: 
Bot: Oh, mojitos are delicious. I


In [307]:
import random
from datasets import load_dataset

def get_random_conv_id(pool, num_conv):
    random_values = random.sample(pool, num_conv)

    for value in random_values:
        pool.remove(value)
    
    return random_values

def extract_format_data(dataset, conv_id):
    dialog = dataset.filter(lambda example:example['conv_id']== conv_id)

    persona = dialog['train']['personality'][-1]
    history_convo = dialog['train']['history'][-1]
    usr_response = dialog['train']['candidates'][-1][-1]

    #preprocess the persona
    persona_processed = "User Persona: \n"
    for sen in persona:
        persona_processed += sen + "\n"

    #preprocess the history conversation
    history_convo_processed = "Dialogue history: \n"
    
    #concat all history except the last utter from the bot
    for i in range(0,len(history_convo)-1,2):
        bot_uttr = "Bot: " + history_convo[i]
        user_uttr = "User: " + history_convo[i+1]
        full_uttr = bot_uttr + "\n" + user_uttr + "\n"
        history_convo_processed += full_uttr

    #add the last utter from bot
    history_convo_processed += "Bot: " + history_convo[-1] + "\n"

    #preprocess the user response
    usr_response_processed = "User: " + usr_response + "\n"
    return [persona_processed, history_convo_processed, usr_response_processed]

# materials[0] is persona, materials[1] is history conversation, materials[2] is user response
def create_example(dataset, conv_id, implicit=False):
    materials = extract_format_data(dataset, conv_id)
    if implicit:
        return materials[1] + materials[2]
    else:
        return materials[0] + materials[1] + materials[2]
    

def create_few_shot_examples(dataset, conv_id, few_shot_no, implicit=False):
    max_conv_id = dataset['train']['conv_id'][-1]
    pool = [num for num in range(1, max_conv_id + 1) if num != conv_id]
    random_conv_ids = get_random_conv_id(pool, few_shot_no)

    few_shot_examples = ""
    for index, conv_id in enumerate(random_conv_ids):
        few_shot_examples += f"Demo {index}:\n" + create_example(dataset, conv_id, implicit) + "\n"
    return few_shot_examples


def construct_prompt(dataset, conv_id, prompt_type, few_shot_no=1, print_output= False):
    max_conv_id = dataset['train']['conv_id'][-1]
    materials = extract_format_data(dataset, conv_id)
    system_prompt = ""
    user_prompt = ""
    few_shot_examples = ""

    # only history dialogue
    if prompt_type == "context_only":
        user_prompt += materials[1] + "User:"

    # task prompt, history dialogue
    if prompt_type == "task_prompt_context_implicit":
        #version 1 Based on the previous conversation history, generate a response for the user that aligns with their profile and the current context of the discussion.
        system_prompt += "Considering the user's profile and the ongoing discussion's context as established in the previous dialogue history, craft a response that is coherent, relevant, and tailored to the user's interests and style of communication."
        user_prompt += materials[1] + "User:"

    # task prompt, persona, and history dialogue
    if prompt_type == "task_prompt_context_explicit":
        system_prompt += "Given the user's profile as outlined in the provided persona information, and considering the context of the ongoing discussion from the previous dialogue history, craft a response that is specifically tailored to resonate with the user's explicit characteristics and maintains the continuity of the dialogue."
        user_prompt += materials[0]+ materials[1] + "User:"

    # few-shot demos, history dialogue
    if prompt_type == "few_shot_implicit":
        system_prompt += "Considering the various user profiles and styles depicted in the provided few-shot examples, and the ongoing discussion's context as established in the previous dialogue history, synthesize a coherent and relevant response. This response should be adaptable to the general preferences and communication styles observed in the examples, while seamlessly continuing the dialogue."

        #set the implicit flag to True to exclude persona in few-shot demos
        few_shot_examples = create_few_shot_examples(dataset, conv_id, few_shot_no, implicit=False)
        user_prompt += few_shot_examples + materials[1]+ "User:"

    if print_output:
        print("### TASK PROMPT ###\n" + system_prompt)
        print("### USER PROMPT ###\n" + user_prompt)
    
    return system_prompt, user_prompt, materials[2]


#context_only, task_prompt_context_implicit, task_prompt_context_explicit, few_shot_implicit
def test():
    dataset = load_dataset("bavard/personachat_truecased", "full")
    result = construct_prompt(dataset, 174, "few_shot_implicit",print_output=True)
    return result
results = test()

Filter: 100%|██████████| 7801/7801 [00:00<00:00, 53247.29 examples/s]

### TASK PROMPT ###
Considering the various user profiles and styles depicted in the provided few-shot examples, and the ongoing discussion's context as established in the previous dialogue history, synthesize a coherent and relevant response. This response should be adaptable to the general preferences and communication styles observed in the examples, while seamlessly continuing the dialogue.
### USER PROMPT ###
Demo 0:
Dialogue history: 
Bot: Hi how are you today?
User: Been working all day. Am a mortician. How about you?
Bot: Good. That's very interesting. Do you enjoy your work?
User: Honestly I do its like helping ppl one last time. Any hobbies?
Bot: I really like to draw.
User: I have a huge stamp collection. Would love to learn to fly though.
Bot: I could never pilot a plane with my sausage fingers.
User: Do you have a favorite movie? I could watch dirty Harry everyday.
Bot: I love to watch action movies.
User: Same! What do you do for work?
Bot: I am a comedian. I try to make 

In [301]:
results[1]

"Dialogue history: \nBot: Hi, how are you doing today?\nUser: I am spending time with my 4 sisters what are you up to.\nBot: Wow, four sisters. Just watching game of thrones.\nUser: That is a good show I watch that while drinking iced tea.\nBot: I agree. What do you do for a living?\nUser: I'm a researcher I'm researching the fact that mermaids are real.\nBot: Interesting. I'm a website designer. Pretty much spend all my time on the computer.\nUser: That's cool my mom does the same thing.\nBot: That's awesome. I have always had a love for technology.\nUser: Tell me more about yourself.\nBot: I really enjoy free diving, how about you, have any hobbies?\nUser: I enjoy hanging with my mother she's my best friend.\nBot: That's nice. Moms are pretty cool too.\nUser:"